In [151]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import os
import warnings


In [152]:
# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn.cluster._kmeans")
warnings.filterwarnings("ignore", message="KMeans is known to have a memory leak on Windows with MKL")

# Adjust this environment variable to control OpenMP's threading
os.environ["OMP_NUM_THREADS"] = "1"

In [153]:
# Load the data into a Pandas DataFrame
market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [154]:
# Generate summary statistics
market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [155]:
# Plot your data to see what's in your DataFrame
market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [156]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_scaled = StandardScaler().fit_transform(df_market_data)

market_scaled[0:5]


array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [157]:
# Create a DataFrame with the scaled data
market_data_scaled = pd.DataFrame(
    # Copy the crypto names from the original data
    data=market_scaled,
    columns=market_data.columns,
    # Set the coinid column as index
    index=market_data.index
)

# Display sample data
market_data_scaled.head(10)


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351


---

### Find the Best Value for k Using the Original Data.

In [158]:
# Create a list with the number of k-values from 1 to 11
k_range = range(1, 11)

In [159]:
# Create an empty list to store the inertia values
inertia = []


for each_k in k_range:
    # Create a KMeans model using the current value of k and set a random_state for reproducibility
    model = KMeans(n_clusters=each_k, random_state=0)
    
    # Fit the model to the scaled market data
    model.fit(market_scaled)
    
    # Append the model's inertia (sum of squared distances of samples to their closest cluster center) to the inertia list
    inertia.append(model.inertia_)


In [160]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": list(k_range), "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow = pd.DataFrame(elbow_data)
elbow.head(10)


,k,inertia
0,1,287.000000
1,2,198.571818
2,3,123.190482
3,4,79.022435
4,5,65.302379
5,6,52.101806
6,7,46.035250
7,8,37.284807
8,9,32.161417
9,10,28.075253


In [161]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)


:Curve   [k]   (inertia)

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [162]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)


In [163]:
# Fit the K-Means model using the scaled data
model.fit(market_scaled)


KMeans(n_clusters=4, random_state=0)

In [164]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_clstr = model.predict(market_scaled)


# Print the resulting array of cluster values.
crypto_clstr

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 0, 0, 0])

In [165]:
# Create a copy of the DataFrame
cluster = market_scaled.copy()


In [166]:
# Convert the scaled data back into a DataFrame
cluster = pd.DataFrame(cluster, columns=market_data.columns, index=market_data.index)

# Add a new column to the DataFrame with the predicted clusters
cluster['crypto_clusters'] = crypto_clstr


# Display sample data
cluster.head(10)


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,crypto_clusters
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533,2
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284,2
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428,2
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351,2


In [167]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

scatter = cluster.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="crypto_clusters",
    hover_cols=["coin_id"]
)

scatter


:NdOverlay   [crypto_clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [168]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)


In [169]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
crypto_pca = pca.fit_transform(market_scaled)

# View the first five rows of the DataFrame. 
crypto_pca[0:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [170]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_


array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** Approximately 89.4% of the total is captured by three PCA Variables.

In [171]:
# Create a new DataFrame with the PCA data.
crypto_pca = pd.DataFrame(
    # Creating a DataFrame with the PCA data
    data=crypto_pca,
    columns=["PC1", "PC2", "PC3"],
    # Copy the crypto names from the original data
    # Set the coinid column as index
    index=market_data.index
)

# Display sample data
crypto_pca.head(10)


,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715
binancecoin,-0.516534,1.388377,0.804071
chainlink,-0.450711,0.517699,2.846143
cardano,-0.345600,0.729439,1.478013
litecoin,-0.649468,0.432165,0.600303


---

### Find the Best Value for k Using the PCA Data

In [172]:
# Create a list with the number of k-values from 1 to 11
k_range2 = list(range(1, 11))


In [173]:
# Create an empty list to store the inertia values
pca_inertia = []


for each_k in k_range2:
    # Create a KMeans model using the current value of k and set a random_state for reproducibility
    pca_model = KMeans(n_clusters=each_k, random_state=0) 
    
    # Fit the model to the scaled market data
    pca_model.fit(crypto_pca)
    
    # Append the model's inertia (sum of squared distances of samples to their closest cluster center) to the inertia list
    pca_inertia.append(pca_model.inertia_) 

In [174]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow = {"k": list(k_range2), "inertia": pca_inertia}

# Create a DataFrame with the data to plot the Elbow curve
pca_elbow = pd.DataFrame(pca_elbow)
pca_elbow.head()

,k,inertia
0,1,256.874086
1,2,168.811896
2,3,93.774626
3,4,49.665497
4,5,37.839466


In [175]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

original_elbow = elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Original Elbow Plot",
    xticks=k
)

# Create individual plots
pca_elbow = pca_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="PCA Elbow Plot",
    xticks=k
)

# Display side by side using the + operator
plots = original_elbow + pca_elbow

# Show the combined plot
plots


:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4 is still the best K value when using the PCA data


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No. The best K value is visually 4 for both. However, in the original elbow plot the inertia at 4 was 79.02 and in the PCA plot it is 49.66 which should yield a better result.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [176]:
# Initialize the K-Means model using the best value for k
pca_model = KMeans(n_clusters=4, random_state=0)



In [177]:
# Fit the K-Means model using the PCA data
pca_model.fit(crypto_pca)

KMeans(n_clusters=4, random_state=0)

In [178]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_crypto_clusters = pca_model.predict(crypto_pca)

# Print the resulting array of cluster values.
pca_crypto_clusters


array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 0, 0, 0])

In [179]:
# Create a copy of the DataFrame with the PCA data
pca_cluster = crypto_pca.copy()


# Add a new column to the DataFrame with the predicted clusters
pca_cluster["crypto_clusters"] = pca_crypto_clusters


# Display sample data
pca_cluster.head(10)


,PC1,PC2,PC3,crypto_clusters
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2
binancecoin,-0.516534,1.388377,0.804071,2
chainlink,-0.450711,0.517699,2.846143,2
cardano,-0.345600,0.729439,1.478013,2
litecoin,-0.649468,0.432165,0.600303,2


In [180]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

pca_scatter = pca_cluster.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="crypto_clusters",
    hover_cols=["coin_id"]
)

pca_scatter


:NdOverlay   [crypto_clusters]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [181]:
# Composite plot to contrast the Elbow curves
plots = original_elbow + pca_elbow
plots

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [182]:
# Composite plot to contrast the clusters
cluster_plots = scatter + pca_scatter
cluster_plots

:Layout
   .NdOverlay.I  :NdOverlay   [crypto_clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [crypto_clusters]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** When using fewer featues there seems to be more association in the clusters. The PCA (Principal Component Analysis) provided a faithful representation of the data clustering compared to the original dataset. It grouped the data into clusters in a manner that was both expected and representative, effectively capturing the underlying structure of the data while maintaining the integrity of the clustered groups.